<a href="https://colab.research.google.com/github/rondinell/Intelig-ncia-Artificial/blob/main/CopaTrade3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ta
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np
import ta # Biblioteca de Análise Técnica

# --- Configuração dos Ativos ---
configuracao_ativos = {
    "Mini Índice":    {"ticker": "^BVSP", "max_contratos": 10},
    "Mini Dólar":     {"ticker": "BRL=X", "max_contratos": 5},
    "Bitcoin Futuro": {"ticker": "BTC-USD", "max_contratos": 20},
    "Ether Futuro":   {"ticker": "ETH-USD", "max_contratos": 10},
    "Solana Futuro":  {"ticker": "SOL-USD", "max_contratos": 10},
    "Ouro Futuro":    {"ticker": "GC=F", "max_contratos": 5},
    "Euro Futuro":    {"ticker": "EURBRL=X", "max_contratos": 5}
}

# --- Função de Análise DIÁRIA (Original) ---
def analisar_ativo(ticker):
    """
    Busca os dados de um ativo, calcula múltiplos indicadores (MM, RSI, MACD, ATR)
    e retorna uma recomendação baseada na confluência de sinais.
    """
    try:
        dados = yf.download(ticker, period="90d", interval="1d", progress=False, auto_adjust=False)
        if dados.empty or len(dados) < 30:
            return {"Preço": "N/A", "RSI": "N/A", "ATR": "N/A", "Recomendação": "Dados insuficientes"}
        close_prices = dados['Close'].squeeze().astype(float)
        high_prices = dados['High'].squeeze().astype(float)
        low_prices = dados['Low'].squeeze().astype(float)
        ultimo_preco = close_prices.iloc[-1]
        mm_curta = close_prices.ewm(span=9, adjust=False).mean().iloc[-1]
        mm_longa = close_prices.ewm(span=21, adjust=False).mean().iloc[-1]
        rsi = ta.momentum.RSIIndicator(close_prices, window=14).rsi().iloc[-1]
        macd_indicador = ta.trend.MACD(close_prices)
        macd_linha = macd_indicador.macd().iloc[-1]
        macd_sinal = macd_indicador.macd_signal().iloc[-1]
        atr = ta.volatility.AverageTrueRange(high=high_prices, low=low_prices, close=close_prices, window=14).average_true_range().iloc[-1]

        recomendacao = "NEUTRO / MONITORAR"
        if (mm_curta > mm_longa) and (macd_linha > macd_sinal) and (rsi > 50 and rsi < 70):
            recomendacao = "COMPRA FORTE (Tendência e Momentum de Alta)"
        elif (mm_curta < mm_longa) and (macd_linha < macd_sinal) and (rsi < 50 and rsi > 30):
            recomendacao = "VENDA FORTE (Tendência e Momentum de Baixa)"
        elif (rsi < 30) and (macd_linha > macd_sinal):
            recomendacao = "MONITORAR COMPRA (Possível Reversão de Baixa)"
        elif (rsi > 70) and (macd_linha < macd_sinal):
            recomendacao = "MONITORAR VENDA (Possível Reversão de Alta)"

        return {"Preço": f"{ultimo_preco:.2f}", "RSI": f"{rsi:.2f}", "ATR": f"{atr:.2f}", "Recomendação": recomendacao}
    except Exception as e:
        return {"Preço": "N/A", "RSI": "N/A", "ATR": "N/A", "Recomendação": f"ERRO: {str(e)}"}

# --- NOVA FUNÇÃO DE ANÁLISE INTRADIÁRIA ---
def analisar_volume_intraday(ticker, nome_ativo):
    """
    Analisa o volume e a pressão de compra/venda intradiária para um ativo.
    """
    print(f"\n--- Análise de Comportamento Intradiário para: {nome_ativo} ({ticker}) ---")
    try:
        dados = yf.download(ticker, period="30d", interval="1h", progress=False, auto_adjust=True)
        if dados.empty:
            print("Não foram encontrados dados intradiários para este ativo.")
            return

        # Filtra para horários de maior liquidez (ajustável)
        dados = dados.between_time('09:00', '21:00')
        if dados.empty:
            print("Não foram encontrados dados dentro do horário de análise (09h-21h).")
            return

        dados['pressao_volume'] = dados['Volume'] * (dados['Close'] - dados['Open'])
        dados['hora'] = dados.index.hour
        analise_por_hora = dados.groupby('hora')['pressao_volume'].mean()

        max_abs_value = analise_por_hora.abs().max()
        if max_abs_value == 0:
            analise_normalizada = analise_por_hora
        else:
            analise_normalizada = analise_por_hora / max_abs_value

        print("Comportamento Médio por Hora (Positivo = Compra | Negativo = Venda)")
        print("-" * 60)
        for hora, pressao in analise_normalizada.items():
            barra = "C" * int(pressao * 25) if pressao > 0 else "V" * int(abs(pressao) * 25)
            print(f" {hora:02d}:00 | {pressao:+.2f} | {barra}")
        print("-" * 60)

    except Exception as e:
        print(f"Ocorreu um erro ao analisar {ticker}: {e}")

# --- Roda as Duas Análises ---
if __name__ == "__main__":
    # --- SECÇÃO 1: Roda a análise diária principal ---
    print("="*60)
    print("      ANALISTA DE ATIVOS - COPA TRADE BTG (V8 Robusto)")
    print(f"      Análise gerada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    print("="*60)

    resultados = []
    for nome, config in configuracao_ativos.items():
        print(f"-> Analisando (Diário) {nome} ({config['ticker']})...")
        analise = analisar_ativo(config['ticker'])
        resultados.append({
            "Ativo": nome,
            "Preço Atual": analise["Preço"],
            "Volatilidade (ATR)": analise["ATR"],
            "RSI(14)": analise["RSI"],
            "Recomendação de Ação": analise["Recomendação"]
        })

    painel = pd.DataFrame(resultados)
    print("\n--- PAINEL DE ANÁLISE TÉCNICA (DIÁRIA) ---")
    pd.set_option('display.max_colwidth', None)
    print(painel.to_string(index=False))

    # --- NOVA SECÇÃO: Executa a análise de volume intradiário ---
    print("\n\n" + "="*60)
    print("      ANÁLISE DE COMPORTAMENTO INTRADIÁRIO (ÚLTIMOS 30 DIAS)")
    print("="*60)

    for nome, config in configuracao_ativos.items():
        analisar_volume_intraday(config['ticker'], nome)

    print("\n\nLembre-se: Esta é uma ferramenta de apoio. Opere com consciência e gerencie seus riscos.")

      ANALISTA DE ATIVOS - COPA TRADE BTG (V8 Robusto)
      Análise gerada em: 08/10/2025 18:03:38
-> Analisando (Diário) Mini Índice (^BVSP)...
-> Analisando (Diário) Mini Dólar (BRL=X)...
-> Analisando (Diário) Bitcoin Futuro (BTC-USD)...
-> Analisando (Diário) Ether Futuro (ETH-USD)...
-> Analisando (Diário) Solana Futuro (SOL-USD)...
-> Analisando (Diário) Ouro Futuro (GC=F)...
-> Analisando (Diário) Euro Futuro (EURBRL=X)...

--- PAINEL DE ANÁLISE TÉCNICA (DIÁRIA) ---
         Ativo Preço Atual Volatilidade (ATR) RSI(14)                        Recomendação de Ação
   Mini Índice   141887.00            1525.20   44.19                          NEUTRO / MONITORAR
    Mini Dólar        5.35               0.05   48.65                          NEUTRO / MONITORAR
Bitcoin Futuro   124093.41            2778.63   66.05 COMPRA FORTE (Tendência e Momentum de Alta)
  Ether Futuro     4539.22             174.09   55.86 COMPRA FORTE (Tendência e Momentum de Alta)
 Solana Futuro      227.47     